# Baseball Prediction: 3b - Augment DataFrame with Odds Data
In the previous notebook we got historical odds data from oddsshark.com and saved them as a set of csv files (with a particular naming convention).

In this notebook we will load that data and augment our primary (game-level) data frame so that it includes this odds data - specifically, the implied probabilities and the over/under, for each game.

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import lxml
import html5lib
%matplotlib inline
from urllib.request import urlopen
import time
import structureboost as stb
import ml_insights as mli
pd.set_option('display.max_columns',1000)
pd.set_option('display.max_rows',1000)

# Load in our game level data

In [2]:
df=pd.read_csv('df_bp1.csv')

/var/folders/9d/8vjj2hs92bq3s8th__4v3jfh0000gn/T/ipykernel_60489/3742704531.py:1: DtypeWarning: Columns (13,14,15,85,87) have mixed types. Specify dtype option on import or set low_memory=False.
  df=pd.read_csv('df_bp1.csv')


# Plan of attack
- Create a dictionary structure to enable us to easily go to a specific game for a particular team and season.
- Iterate through the rows of our main dataframe, look up the home and visiting team for that game to get their odds. (Note: need two different lookups per game)

## Create a dictionary for easy lookup

In [3]:
# Use the same mapping as before
# (better to save to file...)

oddsshark_num_to_team_dict = {}
oddsshark_num_to_team_dict[26995]='PHI'
oddsshark_num_to_team_dict[26996]='SDN'
oddsshark_num_to_team_dict[26997]='SFN'
oddsshark_num_to_team_dict[26998]='ANA'
oddsshark_num_to_team_dict[26999]='DET'
oddsshark_num_to_team_dict[27000]='CIN'
oddsshark_num_to_team_dict[27001]='NYA'
oddsshark_num_to_team_dict[27002]='TEX'
oddsshark_num_to_team_dict[27003]='TBA'
oddsshark_num_to_team_dict[27004]='COL'
oddsshark_num_to_team_dict[27005]='MIN'
oddsshark_num_to_team_dict[27006]='KCA'
oddsshark_num_to_team_dict[27007]='ARI'
oddsshark_num_to_team_dict[27008]='BAL'
oddsshark_num_to_team_dict[27009]='ATL'
oddsshark_num_to_team_dict[27010]='TOR'
oddsshark_num_to_team_dict[27011]='SEA'
oddsshark_num_to_team_dict[27012]='MIL'
oddsshark_num_to_team_dict[27013]='PIT'
oddsshark_num_to_team_dict[27014]='NYN'
oddsshark_num_to_team_dict[27015]='LAN'
oddsshark_num_to_team_dict[27016]='OAK'
oddsshark_num_to_team_dict[27017]='WAS'
oddsshark_num_to_team_dict[27018]='CHA'
oddsshark_num_to_team_dict[27019]='SLN'
oddsshark_num_to_team_dict[27020]='CHN'
oddsshark_num_to_team_dict[27021]='BOS'
oddsshark_num_to_team_dict[27022]='MIA'
oddsshark_num_to_team_dict[27023]='HOU'
oddsshark_num_to_team_dict[27024]='CLE'

In [5]:
# Use the saved files to get the odds information
# We create a dict based on team and season for easy lookup
df_odds_dict={}
for i in range(26995, 27025):
    team_name = oddsshark_num_to_team_dict[i]
    df_odds_dict[team_name] = {}
    print(team_name)
    for season in range(2019,2024):
        fname = 'oddsshark_files/oddsshark_'+team_name+'_'+str(season)+'.csv'
        df_temp = pd.read_csv(fname)
        df_temp['date_dblhead'] = (df_temp.date_numeric.astype(str) + df_temp.dblheader_num.astype(str)).astype(int)
        df_temp.set_index('date_dblhead', inplace=True)
        df_odds_dict[team_name][season] = df_temp

PHI
SDN
SFN
ANA
DET
CIN
NYA
TEX
TBA
COL
MIN
KCA
ARI
BAL
ATL
TOR
SEA
MIL
PIT
NYN
LAN
OAK
WAS
CHA
SLN
CHN
BOS
MIA
HOU
CLE


## Augment our main DataFrame

In [6]:
# Again, we iterate through the main dataframe
# get the team, season, game and then get
# the relevant info from the odds dictionary

implied_prob_h = np.zeros(df.shape[0])
implied_prob_v = np.zeros(df.shape[0])
over_under = np.zeros(df.shape[0])
ou_result = np.full(df.shape[0],'', dtype=object)
for ind, row in df.iterrows():
    if (ind%1000)==0:
        print(ind)
    if row.season<2019:
        continue
    else:
        season = row['season']
        home_team = row['team_h']
        visit_team = row['team_v']
        home_game_no = row['game_no_h']
        visit_game_no = row['game_no_v']
        date_dblh = row['date_dblhead']
        try:
            implied_prob_h[ind] = df_odds_dict[home_team][season].loc[date_dblh,'prob_implied']
            over_under[ind] = df_odds_dict[home_team][season].loc[date_dblh,'Total']
            ou_result[ind] = df_odds_dict[home_team][season].loc[date_dblh,'OU']
        except KeyError:
            print(f'Game not found wrt home_team:{home_team} vs {visit_team} date_dbl {date_dblh}')
        try:
            implied_prob_v[ind] = df_odds_dict[visit_team][season].loc[date_dblh,'prob_implied']
        except KeyError:
            print(f'Game not found wrt visit_team:{visit_team} vs {home_team} date_dbl {date_dblh}')

0
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
23000
24000
25000
26000
27000
28000
29000
30000
31000
32000
33000
34000
35000
36000
37000
38000
39000
40000
41000
42000
43000
44000
45000
46000
47000
48000
49000
50000
51000
52000
53000
54000
55000
56000
57000
58000
59000
60000
61000
62000
63000
64000
65000
66000
67000
Game not found wrt home_team:OAK vs SEA date_dbl 201903200
Game not found wrt visit_team:SEA vs OAK date_dbl 201903200
Game not found wrt home_team:OAK vs SEA date_dbl 201903210
Game not found wrt visit_team:SEA vs OAK date_dbl 201903210
Game not found wrt home_team:CIN vs PIT date_dbl 201903280
Game not found wrt visit_team:PIT vs CIN date_dbl 201903280
Game not found wrt home_team:LAN vs ARI date_dbl 201903280
Game not found wrt visit_team:ARI vs LAN date_dbl 201903280
Game not found wrt home_team:MIA vs COL date_dbl 201903280
Game not found wrt visit_team:COL vs MIA date_dbl 201903280
Game not f

In [7]:
df['implied_prob_h'] = implied_prob_h
df['implied_prob_v'] = implied_prob_v
df['implied_prob_h_mid'] = (implied_prob_h + (1-implied_prob_v))/2
df['over_under_line']=over_under
df['over_under_result']=ou_result

## Drop 3 games with no odds

In [8]:
df[(df.season>=2019) & (df.implied_prob_h==0)]

,date,dblheader_code,day_of_week,team_v,league_v,game_no_v,team_h,league_h,game_no_h,runs_v,runs_h,outs_total,day_night,completion_info,forfeit_info,protest_info,ballpark_id,attendance,game_minutes,linescore_v,linescore_h,AB_v,H_v,2B_v,3B_v,HR_v,RBI_v,SH_v,SF_v,HBP_v,BB_v,IBB_v,SO_v,SB_v,CS_v,GIDP_v,CI_v,LOB_v,P_num_v,ERind_v,ERteam_v,WP_v,balk_v,PO_v,ASST_v,ERR_v,PB_v,DP_v,TP_v,AB_h,H_h,2B_h,3B_h,HR_h,RBI_h,SH_h,SF_h,HBP_h,BB_h,IBB_h,SO_h,SB_h,CS_h,GIDP_h,CI_h,LOB_h,P_num_h,ERind_h,ERteam_h,WP_h,balk_h,PO_h,ASST_h,ERR_h,PB_h,DP_h,TP_h,ump_HB_id,ump_HB_name,ump_1B_id,ump_1B_name,ump_2B_id,ump_2B_name,ump_3B_id,ump_3B_name,ump_LF_id,ump_LF_name,ump_RF_id,ump_RF_name,mgr_id_v,mgr_name_v,mgr_id_h,mgr_name_h,pitcher_id_w,pitcher_name_w,pitcher_id_l,pitcher_name_l,pitcher_id_s,pitcher_name_s,GWRBI_id,GWRBI_name,pitcher_start_id_v,pitcher_start_name_v,pitcher_start_id_h,pitcher_start_name_h,batter1_name_v,batter1_id_v,batter1_pos_v,batter2_name_v,batter2_id_v,batter2_pos_v,batter3_name_v,batter3_id_v,batter3_pos_v,batter4_name_v,batter4_id_v,batter4_pos_v,batter5_name_v,batter5_id_v,batter5_pos_v,batter6_name_v,batter6_id_v,batter6_pos_v,batter7_name_v,batter7_id_v,batter7_pos_v,batter8_name_v,batter8_id_v,batter8_pos_v,batter9_name_v,batter9_id_v,batter9_pos_v,batter1_name_h,batter1_id_h,batter1_pos_h,batter2_name_h,batter2_id_h,batter2_pos_h,batter3_name_h,batter3_id_h,batter3_pos_h,batter4_name_h,batter4_id_h,batter4_pos_h,batter5_name_h,batter5_id_h,batter5_pos_h,batter6_name_h,batter6_id_h,batter6_pos_h,batter7_name_h,batter7_id_h,batter7_pos_h,batter8_name_h,batter8_id_h,batter8_pos_h,batter9_name_h,batter9_id_h,batter9_pos_h,misc_info,acqui_info,season,run_diff,home_victory,run_total,date_dblhead,BATAVG_162_h,BATAVG_162_v,OBP_162_h,OBP_162_v,SLG_162_h,SLG_162_v,OBS_162_h,OBS_162_v,SB_162_h,SB_162_v,CS_162_h,CS_162_v,ERR_162_h,ERR_162_v,BATAVG_30_h,BATAVG_30_v,OBP_30_h,OBP_30_v,SLG_30_h,SLG_30_v,OBS_30_h,OBS_30_v,SB_30_h,SB_30_v,CS_30_h,CS_30_v,ERR_30_h,ERR_30_v,implied_prob_h,implied_prob_v,implied_prob_h_mid,over_under_line,over_under_result
67753,20190320,0,Wed,SEA,AL,1,OAK,AL,1,9,7,54,N,NaN,NaN,NaN,TOK01,45787.0,204,005130000,112000300,31,7,1,0,2,9,1,1,2,6,0,10,2,1,0,0,5,5,6,6,1,0,27,11,1,0,2,0,35,9,3,0,3,7,0,0,0,3,0,8,0,0,2,0,4,7,9,9,2,0,27,4,0,0,0,0,nelsj901,Jeff Nelson,gibsh902,Tripp Gibson,barkl901,Lance Barksdale,muchm901,Mike Muchlinski,NaN,(none),NaN,(none),servs002,Scott Servais,melvb001,Bob Melvin,gonzm005,Marco Gonzales,fierm001,Michael Fiers,strih001,Hunter Strickland,santd002,Domingo Santana,gonzm005,Marco Gonzales,fierm001,Michael Fiers,gordd002,Dee Strange-Gordon,4,hanim001,Mitch Haniger,8,brucj001,Jay Bruce,3,encae001,Edwin Encarnacion,10,santd002,Domingo Santana,7,narvo001,Omar Narvaez,2,healr001,Ryon Healy,5,beckt001,Tim Beckham,6,suzui001,Ichiro Suzuki,9,laurr001,Ramon Laureano,8,chapm001,Matt Chapman,5,piscs001,Stephen Piscotty,9,davik003,Khris Davis,10,pindc001,Chad Pinder,7,olsom001,Matt Olson,3,profj001,Jurickson Profar,4,semim001,Marcus Semien,6,hundn001,Nick Hundley,2,NaN,Y,2019,-2,0,16,201903200,0.252196,0.254308,0.319302,0.308262,0.439147,0.408126,0.758448,0.716388,35.0,79.0,21.0,37.0,88.0,88.0,0.256836,0.248031,0.336530,0.312950,0.444336,0.399606,0.780866,0.712556,5.0,10.0,2.0,5.0,13.0,19.0,0.0,0.0,0.5,0.0,
67754,20190321,0,Thu,SEA,AL,2,OAK,AL,2,5,4,72,N,NaN,NaN,NaN,TOK01,46451.0,267,021000100001,000020200000,43,9,4,0,2,5,0,1,2,5,0,9,1,0,1,0,10,7,3,3,2,0,36,12,1,0,1,0,46,10,1,0,0,3,0,0,0,6,0,10,0,0,1,0,12,8,5,5,0,1,36,8,0,0,1,0,welkb901,Bill Welke,barkl901,Lance Barksdale,muchm901,Mike Muchlinski,nelsj901,Jeff Nelson,NaN,(none),NaN,(none),servs002,Scott Servais,melvb001,Bob Melvin,rossz001,Zac Rosscup,buchr001,Ryan Buchter,strih001,Hunter Strickland,santd002,Domingo Santana,kikuy001,Yusei Kikuchi,estrm001,Marco Estrada,gordd002,Dee Strange-Gordon,4,hanim001,Mitch Haniger,8,brucj001,Jay Bruce,3,encae001,Edwin Encarnacion,10,santd002,Domingo Santana,7,narvo001,Omar Narvaez,2,beckt001,Tim Beckham,6,

In [9]:
indices_to_drop = df[(df.season>=2019) & (df.implied_prob_h==0)].index
indices_to_drop

Index([67753, 67754, 67755, 67756, 67757, 67758, 67759, 67760, 67761, 67762,
       ...
       70175, 70176, 70177, 70178, 70179, 70180, 70181, 72218, 72501, 72502],
      dtype='int64', length=2432)

In [10]:
df.shape

(75939, 199)

In [11]:
df.drop(indices_to_drop, inplace=True)
df.shape

(73507, 199)

In [12]:
df.reset_index(inplace=True, drop=True)
df.shape

(73507, 199)

In [13]:
df.to_csv('df_bp3.csv', index=False)